In [6]:
import numpy as np
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer


from sklearn.datasets import fetch_20newsgroups

categories = ['comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
              'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey']

newsgroups_data = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

In [3]:
#QUESTION 1
def tokenize(text):
    tokens = nltk.tokenize.word_tokenize(text)
    tokens = [token.strip(string.punctuation) for token in tokens if token.isalnum()]
    return tokens

tfidfVectorizer = TfidfVectorizer(min_df=3, stop_words='english', tokenizer = tokenize)
tfidfVectors = tfidfVectorizer.fit_transform(newsgroups_data.data)
print (tfidfVectors.shape[0], tfidfVectors.shape[1])

7882 15728


In [5]:
#QUESTION 2 AND 3
from sklearn.cluster import KMeans
from sklearn import metrics

vectorFunc = np.vectorize(lambda x : int(x / 4))
labels = vectorFunc(newsgroups_data.target)

cur_k = 2
km = KMeans(n_clusters = cur_k, init = 'k-means++')
km.fit(tfidfVectors)

def contingency_table(true_labels, predicted_labels):
    n_clusters = len(np.unique(predicted_labels))
    CT = np.zeros(shape = (n_clusters,n_clusters))
    uniq_true = np.unique(true_labels)
    for i, true_label in enumerate(uniq_true):
        for j, predicted_label in enumerate(predicted_labels):
            if(true_labels[j] == true_label):
                CT[i][predicted_label] += 1
    return CT

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
%matplotlib inline

def evaluate(labels, kmeans_labels_):
    print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, kmeans_labels_))
    print("Completeness: %0.3f" % metrics.completeness_score(labels, kmeans_labels_))
    print("V-measure: %0.3f" % metrics.v_measure_score(labels, kmeans_labels_))
    print("Adjusted Rand-Index: %.3f" % metrics.adjusted_rand_score(labels, kmeans_labels_))
    print("Adjusted Mutual Information Score: %.3f" % metrics.adjusted_mutual_info_score(labels, kmeans_labels_))
    print("Contingency Table: ")
    CT = contingency_table(labels, kmeans_labels_).astype(int)
    if len(np.unique(labels)) > 2:
        print(plt.matshow(A, cmap=get_cmap('Blues')))
    else:
        print(CT)

evaluate(labels, km.labels_)

Homogeneity: 0.389
Completeness: 0.435
V-measure: 0.411
Adjusted Rand-Index: 0.380
Adjusted Mutual Information Score: 0.389
Contingency Table: 
[[1482 2421]
 [3951   28]]


C:\Users\laksh\Anaconda3\lib\site-packages\sklearn\metrics\cluster\supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
